In [2]:
import pandas as pd
import numpy as np
import pathlib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from model.threshold_guess import compute_thresholds, cut
from model.gosdt import GOSDT
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
import copy
from sklearn.model_selection import GridSearchCV , StratifiedKFold

# H355

  

In [2]:
# read the dataset
df = pd.read_csv("../experiments/datasets/H355.csv")
df.loc[df[df.labels.isin([ 3.0,4.0])].index, 'labels']=2.0
X, y = df.iloc[:,:-1], df.iloc[:,-1]
X= X.drop(['time'], axis=1)
h = X.columns[:-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [3]:
h

Index(['co2', 'power1', 'power3', 'power4', 'power2', 'Temperature_CO2',
       'Humidity_CO2', 'Contact_porte', 'Temperature_1', 'Humidity_1', 'COV',
       'Temperature_12'],
      dtype='object')

In [4]:
X_train

,co2,power1,power3,power4,power2,Temperature_CO2,Humidity_CO2,Contact_porte,Temperature_1,Humidity_1,COV,Temperature_12,Humidity_12
907,410.0,22.0,11.0,0.0,0.0,19.4,27.0,0.0,19.20,33.2,17.0,20.00,34.8
633,780.0,24.0,130.0,0.0,0.0,20.8,27.0,0.0,20.32,32.0,52.0,20.48,30.4
329,1090.0,82.0,0.0,0.0,0.0,20.2,32.0,1.0,20.32,37.6,36.0,19.84,39.2
965,390.0,22.0,11.0,0.0,0.0,18.8,30.0,0.0,18.08,37.6,19.0,19.84,38.0
153,670.0,22.0,11.0,0.0,0.0,18.6,31.0,0.0,18.56,36.8,33.0,18.72,35.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,1540.0,22.0,24.0,0.0,0.0,20.8,32.0,0.0,20.80,38.8,39.0,20.00,42.0
1044,1480.0,73.0,94.0,0.0,0.0,20.6,37.0,1.0,20.48,45.2,48.0,20.64,42.0
90,2550.0,22.0,11.0,0.0,52.0,19.8,37.0,0.0,19.52,44.8,60.0,19.36,42.8
587,390.0,22.0,11.0,0.0,0.0,19.6,28.0,0.0,20.16,32.0,22.0,18.40,37.2


In [ ]:
# train GOSDT model
config = {
            "regularization": 0.01, # regularization penalizes the tree with more leaves. We recommend to set it to relative high value to find a sparse tree.  
            "depth_budget": 6,
            "time_limit": 120, # training time limit in seconds
            "similar_support": False
        }

model = GOSDT(config)

model.fit(X_train, y_train)
print("evaluate the model, extracting tree and scores", flush=True)

acc = model.score(X_train, y_train)
test_acc = model.score(X_test, y_test)
n_leaves = model.leaves()

print("Training accuracy: {}".format(acc))
print("Testing accuracy: {}".format(test_acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

In [ ]:
source_tree_structure = model.tree


In [ ]:
source_leaf_nodes = model.tree.leaves

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

test_accuracy = clf.score(X_test, y_test)
print(test_accuracy)
prediction = clf.predict(X_test)
cm = confusion_matrix(y_test, prediction)
print("Confusion Matrix:\n ", cm)

In the above, we had to constrain our search to very simple trees. We're likely to get a more useful and accurate model when we allow our model to find more complicated trees (we do this with less constraint on depth and with smaller regularization). However, for real-valued datasets like COMPAS, GOSDT might not finish running within a couple minutes. An example is shown below where the depth budget is set to 6 and the regularization is set to 0.001 for the COMPAS dataset. After 60 seconds, GOSDT hits the time limit and returns the current best model. 

In [ ]:
# set GOSDT configuration
# set a smaller regularization and a higher depth budget. All other hyparameters are same as before. 
config = {
            "regularization": 0.001, 
            "depth_budget": 6,
            "time_limit": 60, 
            "similar_support": False
        }

model = GOSDT(config)

model.fit(X, y)
print("evaluate the model, extracting tree and scores", flush=True)

acc = model.score(X, y)
n_leaves = model.leaves()

print("Training accuracy: {}".format(acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

In [ ]:
# read the dataset
df = pd.read_csv("../experiments/datasets/H358_edited.csv")
df.loc[df[df.labels.isin([ 3.0,4.0])].index, 'labels']=2.0
X, y = df.iloc[:,:-1], df.iloc[:,-1]
X= X.drop(['time'], axis=1)
h = X.columns[:-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
# train GOSDT model
config = {
            "regularization": 0.01, # regularization penalizes the tree with more leaves. We recommend to set it to relative high value to find a sparse tree.  
            "depth_budget": 6,
            "time_limit": 120, # training time limit in seconds
            "similar_support": False
        }

model = GOSDT(config)

model.fit(X_train, y_train)
print("evaluate the model, extracting tree and scores", flush=True)

acc = model.score(X_test, y_test)
n_leaves = model.leaves()

print("Test accuracy: {}".format(acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

test_accuracy = clf.score(X_test, y_test)
print(test_accuracy)
prediction = clf.predict(X_test)
cm = confusion_matrix(y_test, prediction)
print("Confusion Matrix:\n ", cm)

With the time-out, GOSDT didn't find a very accurate tree because it couldn't finish optimizing. If it did finish optimizing, we might need to wait for a long time or use a large amount of memory. An alternative, presented in [our AAAI '22 paper](https://arxiv.org/abs/2112.00798), is to use threshold and lowerbound guesses to help GOSDT find a near optimal tree for COMPAS within 1 minute. The guessing process works as follows: 

#### 1. Threshold guess: we first use a black box model, gradient boosted trees, to guess useful thresholds. 

In [ ]:
# GBDT parameters for threshold and lower bound guesses
n_est = 40
max_depth = 1

One of the biggest slowdowns for decision trees is deciding how to binarize the continuous features. We had a separate column in our dataset for age <=20.5, age <=21.5, age <= 22.5, ... and so on. Using threshold guessing lets us prune the thresholds we use, by looking at which thresholds are used by gradient boosted trees. 

In [ ]:
# guess thresholds
X_train = pd.DataFrame(X_train, columns=h)
X_train_guessed, thresholds, header, threshold_guess_time = compute_thresholds(X_train.copy(), y_train, n_est, max_depth)
X_train_guessed

#### 2. Lower bound guess: we use the gradient boosted tree to get labels for the training dataset. 

We determine what labels the gradient boosted tree model would predict for each of the training points. The idea is that many good decision trees are likely to make similar predictions to the gradient boosted tree on the training data.

In [ ]:
# guess lower bound
import time
start_time = time.perf_counter()
clf = GradientBoostingClassifier(n_estimators=n_est, max_depth=max_depth, random_state=42)
clf.fit(X_train_guessed, y_train.values.flatten())
warm_labels = clf.predict(X_train_guessed)

elapsed_time = time.perf_counter() - start_time

lb_time = elapsed_time

In [ ]:
# save the labels as a tmp file and return the path to it.
labelsdir = pathlib.Path('/tmp/warm_lb_labels')
labelsdir.mkdir(exist_ok=True, parents=True)

labelpath = labelsdir / 'warm_label.tmp'
labelpath = str(labelpath)
pd.DataFrame(warm_labels).to_csv(labelpath, header="class_labels",index=None) # TODO: verify this formats correctly for gosdt (shouldn't require headers)


#### 3. train GOSDT model

We can use the information from our guesses to speed up training GOSDT, while retaining some nice properties on the tree it returns (see our AAAI '22 paper for more information). 

In [ ]:
# train GOSDT model
config = {
            "regularization": 0.001, 
            "depth_budget": 6,
            "time_limit": 60, 
            "similar_support": False
        }

model = GOSDT(config)

model.fit(X_train_guessed, pd.DataFrame(y_train))

print("evaluate the model, extracting tree and scores", flush=True)

#### 4. Get results

In [ ]:
# get the results
acc = model.score(X_train_guessed, y_train)
n_leaves = model.leaves()
test_acc = model.score(X_test, y_test)
print("Training accuracy: {}".format(acc))
print("Test accuracy: {}".format(test_acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

In [ ]:
prediction = model.predict(X_test)
cm = confusion_matrix(y_test, prediction)
print("Confusion Matrix:\n ", cm)

After using the threshold and lower bound guesses, GOSDT can find a near optimal tree within seconds. This tree is much more accurate than the tree we found earlier, when using much more restrictive parameters. 

In [112]:
df = pd.read_csv("../experiments/datasets/H355_edited.csv")
df.loc[df[df.labels.isin([ 3.0,4.0])].index, 'labels']=2.0
X, y = df.iloc[:,:-1], df.iloc[:,-1]
X= X.drop(['time'], axis=1)
h = X.columns[:-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)
n_est = 40
max_depth = 1
X_train = pd.DataFrame(X_train, columns=h)
X_test = pd.DataFrame(X_test, columns=h)
X_train_guessed, thresholds, header, threshold_guess_time = compute_thresholds(X_train.copy(), y_train, n_est, max_depth)
X_test_guessed = cut(X_test.copy(), thresholds)
X_test_guessed = X_test_guessed[header]
print(X_train_guessed.shape)
print(X_test_guessed.shape)
print("train set column names == test set column names: {}".format(list(X_train_guessed.columns)==list(X_test_guessed.columns)))

# guess lower bound
import time
start_time = time.perf_counter()
clf = GradientBoostingClassifier(n_estimators=n_est, max_depth=max_depth, random_state=42)
clf.fit(X_train_guessed, y_train.values.flatten())
warm_labels = clf.predict(X_train_guessed)

elapsed_time = time.perf_counter() - start_time

lb_time = elapsed_time
labelsdir = pathlib.Path('/tmp/warm_lb_labels')
labelsdir.mkdir(exist_ok=True, parents=True)

labelpath = labelsdir / 'warm_label.tmp'
labelpath = str(labelpath)
pd.DataFrame(warm_labels).to_csv(labelpath, header="class_labels",index=None)
config = {
            "regularization": 0.001,
            "depth_budget": 5,
            "time_limit": 60,
            "warm_LB": True,
            "path_to_labels": labelpath,
            "similar_support": False
        }

model = GOSDT(config)

model.fit(X_train_guessed, pd.DataFrame(y_train))

print("evaluate the model, extracting tree and scores", flush=True)
train_acc = model.score(X_train_guessed, y_train)
test_acc = model.score(X_test_guessed, y_test)
n_leaves = model.leaves()
n_nodes = model.nodes()
time = model.utime

print("Model training time: {}".format(time))
print("Training accuracy: {}".format(train_acc))
print("Test accuracy: {}".format(test_acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

prediction = model.predict(X_test_guessed)
cm = confusion_matrix(y_test, prediction)
print("Confusion Matrix:\n ", cm)


C:\Users\ASUS-PC\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
C:\Users\ASUS-PC\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
C:\Users\ASUS-PC\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
C:\Users\ASUS-PC\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' wh

(854, 8)
(214, 8)
train set column names == test set column names: True
gosdt reported successful execution
training completed. 0.000/0.000/0.008 (user, system, wall), mem=0 MB
bounds: [0.155738..0.155738] (0.000000) loss=0.148712, iterations=330
evaluate the model, extracting tree and scores
Model training time: 0.0
Training accuracy: 0.8512880562060889
Test accuracy: 0.883177570093458
# of leaves: 6
if co2<=675.0 = 1 and power<=28.0 = 1 then:
    predicted class: 0
    misclassification penalty: 0.016
    complexity penalty: 0.001

else if co2<=675.0 = 1 and power<=28.0 != 1 then:
    predicted class: 1
    misclassification penalty: 0.001
    complexity penalty: 0.001

else if Temperature<=19.800000190734863 = 1 and co2<=675.0 != 1 and power<=28.0 = 1 then:
    predicted class: 0
    misclassification penalty: 0.005
    complexity penalty: 0.001

else if Temperature<=19.800000190734863 = 1 and co2<=675.0 != 1 and power<=28.0 != 1 then:
    predicted class: 1
    misclassification pe

# Simple Transfer Learning
Integrating GOSDT with Simple TL

In [113]:
df = pd.read_csv("../experiments/datasets/H355_edited.csv")
df.loc[df[df.labels.isin([2.0, 3.0,4.0])].index, 'labels']=1.0
Xs, ys = df.iloc[:,:-1], df.iloc[:,-1]
Xs= Xs.drop(['time'], axis=1)
h = Xs.columns[:-1]
Xs.shape


# read the dataset
df = pd.read_csv("../experiments/datasets/H358_edited.csv")
df.loc[df[df.labels.isin([ 2.0,3.0,4.0])].index, 'labels']=1.0
Xt, yt = df.iloc[:,:-1], df.iloc[:,-1]
Xt= Xt.drop(['time'], axis=1)
h = Xt.columns

# If train-test split is desired
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(Xs, ys, test_size=0.2, random_state=2021)
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(Xt, yt, test_size=0.8, random_state=2021)



In [114]:
X_train_t = X_train_t.reset_index(drop=True)
X_train_s = X_train_s.reset_index(drop=True)
y_train_t = y_train_t.reset_index(drop=True)
y_train_s = y_train_s.reset_index(drop=True)

In [115]:
X_train = pd.concat([X_train_s, X_train_t])
y_train = pd.concat([y_train_s, y_train_t])



In [116]:
y_test_s.shape

(214,)

In [117]:
n_est = 40
max_depth = 1
X_train = pd.DataFrame(X_train, columns=h)
X_test = pd.DataFrame(X_test_s, columns=h)
X_train_guessed, thresholds, header, threshold_guess_time = compute_thresholds(X_train.copy(), y_train, n_est, max_depth)
X_test_guessed = cut(X_test_s.copy(), thresholds)
X_test_guessed = X_test_guessed[header]

print("train set column names == test set column names: {}".format(list(X_train_guessed.columns)==list(X_test_guessed.columns)))

X_train_s = X_train_s.reset_index(drop=True)
X_train_s = X_train_guessed.iloc[:854]
y_train_s = y_train.iloc[:854]
X_train_t = X_train_t.reset_index(drop=True)
X_train_t = X_train_guessed.iloc[-288:]
y_train_t = y_train.iloc[-288:]
X_train_s.shape

import time
start_time = time.perf_counter()
clf = GradientBoostingClassifier(n_estimators=n_est, max_depth=max_depth, random_state=42)
clf.fit(X_train_s, y_train_s.values.flatten())
warm_labels = clf.predict(X_train_s)

elapsed_time = time.perf_counter() - start_time

lb_time = elapsed_time
labelsdir = pathlib.Path('/tmp/warm_lb_labels')
labelsdir.mkdir(exist_ok=True, parents=True)

labelpath = labelsdir / 'warm_label.tmp'
labelpath = str(labelpath)
pd.DataFrame(warm_labels).to_csv(labelpath, header="class_labels",index=None)


config = {
            "regularization": 0.001,
            "depth_budget": 10,
            "time_limit": 60,
            "warm_LB": True,
            "path_to_labels": labelpath,
            "similar_support": False
        }

model = GOSDT(config)

model.fit(X_train_s, pd.DataFrame(y_train_s))

print("evaluate the model, extracting tree and scores", flush=True)
train_acc = model.score(X_train_s, y_train_s)
test_acc = model.score(X_test_guessed, y_test_s)
n_leaves = model.leaves()
n_nodes = model.nodes()
time = model.utime

print("Model training time: {}".format(time))
print("Training accuracy: {}".format(train_acc))
print("Test accuracy: {}".format(test_acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

prediction = model.predict(X_test_guessed)
cm = confusion_matrix(y_test_s, prediction)
print("Confusion Matrix:\n ", cm)

C:\Users\ASUS-PC\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
C:\Users\ASUS-PC\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
C:\Users\ASUS-PC\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
C:\Users\ASUS-PC\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' wh

train set column names == test set column names: True
gosdt reported successful execution
training completed. 0.000/0.000/0.026 (user, system, wall), mem=0 MB
bounds: [0.056206..0.056206] (0.000000) loss=0.051522, iterations=997
evaluate the model, extracting tree and scores
Model training time: 0.0
Training accuracy: 0.9484777517564403
Test accuracy: 0.9579439252336449
# of leaves: 4
if Door_contact<=0.5 = 1 and power<=10.625 = 1 then:
    predicted class: 0
    misclassification penalty: 0.015
    complexity penalty: 0.001

else if Door_contact<=0.5 != 1 and power<=10.625 = 1 then:
    predicted class: 1
    misclassification penalty: 0.004
    complexity penalty: 0.001

else if co2<=505.0 = 1 and power<=10.625 != 1 then:
    predicted class: 0
    misclassification penalty: 0.001
    complexity penalty: 0.001

else if co2<=505.0 != 1 and power<=10.625 != 1 then:
    predicted class: 1
    misclassification penalty: 0.032
    complexity penalty: 0.001
Confusion Matrix:
  [[150   9]
 

In [118]:
X_test_guessed_t = cut(X_test_t.copy(), thresholds)
X_test_guessed_t = X_test_guessed_t[header]


In [119]:
X_test_guessed_t = X_test_guessed_t.reset_index(drop=True)
y_test_t = y_test_t.reset_index(drop=True)

In [120]:
import time
start_time = time.perf_counter()
clf = GradientBoostingClassifier(n_estimators=n_est, max_depth=max_depth, random_state=42)
clf.fit(X_train_t, y_train_t.values.flatten())
warm_labels = clf.predict(X_train_t)

elapsed_time = time.perf_counter() - start_time

lb_time = elapsed_time
labelsdir = pathlib.Path('/tmp/warm_lb_labels')
labelsdir.mkdir(exist_ok=True, parents=True)

labelpath = labelsdir / 'warm_label.tmp'
labelpath = str(labelpath)
pd.DataFrame(warm_labels).to_csv(labelpath, header="class_labels",index=None)

model.fit(X_train_t, pd.DataFrame(y_train_t))

print("evaluate the model, extracting tree and scores", flush=True)
train_acc = model.score(X_train_t, y_train_t)
test_acc = model.score(X_test_guessed_t, y_test_t)
n_leaves = model.leaves()
n_nodes = model.nodes()
time = model.utime

print("Model training time: {}".format(time))
print("Training accuracy: {}".format(train_acc))
print("Test accuracy: {}".format(test_acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

prediction = model.predict(X_test_guessed_t)
cm = confusion_matrix(y_test_t, prediction)
print("Confusion Matrix:\n ", cm)

gosdt reported successful execution
training completed. 0.000/0.000/0.004 (user, system, wall), mem=0 MB
bounds: [0.208333..0.208333] (0.000000) loss=0.194444, iterations=321
evaluate the model, extracting tree and scores
Model training time: 0.0
Training accuracy: 0.8055555555555556
Test accuracy: 0.8038194444444444
# of leaves: 4
if Door_contact<=0.5 = 1 and co2<=475.0 = 1 then:
    predicted class: 0
    misclassification penalty: 0.094
    complexity penalty: 0.003

else if Door_contact<=0.5 != 1 and co2<=475.0 = 1 then:
    predicted class: 1
    misclassification penalty: 0.007
    complexity penalty: 0.003

else if co2<=475.0 != 1 and co2<=725.0 = 1 then:
    predicted class: 1
    misclassification penalty: 0.073
    complexity penalty: 0.003

else if co2<=475.0 != 1 and co2<=725.0 != 1 then:
    predicted class: 0
    misclassification penalty: 0.021
    complexity penalty: 0.003
Confusion Matrix:
  [[727 113]
 [113 199]]


As you can see, we find a reasonably accurate model quite quickly on datasets as complicated as FICO!

Thank you for reading our tutorial. Please do try out our methods with different parameters and datasets. Happy tree training!